In [97]:
import pandas as pd
import numpy as np
import math

In [98]:
def PrepareData(x,y) :
    #read image
    data = []
    with open(x) as filehandle:
        lines = filehandle.readlines()
        for line in lines:
                temp = line[:-1]
                data.append(temp)
    
    for i in range(len(data)):
            data[i]=data[i].replace(' ','0')
            data[i]=data[i].replace('#','1')
            data[i]=data[i].replace('+','1')
    df=[]
    x=[]
    str_data =""
    for i in range(len(data)):  
        str_data= str_data + data[i]
        if  len(str_data) == 784 :
            for j in range(len(str_data)):
                df.append(int(str_data[j]))
            matr= np.reshape(df,(28,28))
            x.append(matr)
            df=[]
            str_data=""
    # read label
    label = open(y)
    temp=label.readlines()
    y=[]
    for i in range(len(temp)):
        y.append(int(temp[i]))
    return x,y

In [99]:
x_trn,y_trn=PrepareData("trainingimages","traininglabels")

In [100]:
x_tst,y_tst=PrepareData("testimages","testlabels")

In [101]:
def count_elm(m,lm0,lm1):
    index=y_trn[m]
    for i,j in np.ndindex(x_trn[m].shape):
                if x_trn[m][i,j]  == 0 :
                     lm0[index][i,j] +=1
                else :
                    lm1[index][i,j] +=1
                    
    return lm0,lm1              

In [102]:
def train() :
    count_class=np.array([0,0,0,0,0,0,0,0,0,0])
    class_dict = {i:y_trn.count(i) for i in y_trn}
    for keys in class_dict.keys():
          count_class[keys]=class_dict[keys]
    #print(count_class)        
    m=0
    #create dict
    list_matrix0=[]
    list_matrix1=[]
    for i in range(10):
        list_matrix0.append(np.zeros((28,28),dtype='float64'))
        list_matrix1.append(np.zeros((28,28),dtype='float64'))
    #countf(i,j)    
    while m<5000 :
        list_matrix0,list_matrix1=count_elm(m,list_matrix0,list_matrix1)
        m +=1
    #probf(i,j)
    for item in range(10):
        for i,j in np.ndindex(list_matrix0[item].shape):
            #Laplace Smoothing
            list_matrix0[item][i,j] = (list_matrix0[item][i,j]+0.001) /(count_class[item]+0.002)
    
    for item in range(10):
        for i,j in np.ndindex(list_matrix1[item].shape):
                #list_matrix1[item][i,j] =0.1
                list_matrix1[item][i,j] = (list_matrix1[item][i,j]+0.001) /(count_class[item]+0.002)
            
            
    return  list_matrix0 ,list_matrix1,count_class  

In [103]:
def predict(list_matrix0,list_matrix1,count_class):
    pro_class=[]
    y_hat=[]
    for i in range(10):
        pro_class.append(count_class[i]/5000) 
       
    m=0
    while m <1000:
        y=[]
        for item in range(10):
            sum_pro=0
            
            for i,j in np.ndindex(x_tst[m].shape):
                if x_tst[m][i,j]==0:
                    sum_pro=sum_pro + math.log10(list_matrix0[item][i,j])
             
                else:
                    sum_pro=sum_pro + math.log10(list_matrix1[item][i,j])
            sum_pro = sum_pro + math.log10(pro_class[item]) 
            y.append(sum_pro)
        y=np.array(y)    
        y_hat.append(y.argmax())
        m+=1
    return y_hat    

In [104]:
def predicttrn(list_matrix0,list_matrix1,count_class):
    pro_class=[]
    y_hat=[]
    for i in range(10):
        pro_class.append(count_class[i]/5000) 
       
    m=0
    while m <5000:
        y=[]
        for item in range(10):
            sum_pro=0
            
            for i,j in np.ndindex(x_trn[m].shape):
                if x_trn[m][i,j]==0:
                    sum_pro=sum_pro + math.log10(list_matrix0[item][i,j])
             
                else:
                    sum_pro=sum_pro + math.log10(list_matrix1[item][i,j])
            sum_pro = sum_pro + math.log10(pro_class[item]) 
            y.append(sum_pro)
        y=np.array(y)    
        y_hat.append(y.argmax())
        m+=1
    return y_hat

In [105]:
list_matrix0,list_matrix1,count_class=train()


In [106]:
y_hat = predict(list_matrix0,list_matrix1,count_class)

In [107]:
y_hat=np.array(y_hat)
y_tst=np.array(y_tst)
accuracy_test = np.sum(y_tst == y_hat)/(1000)
print('accuracy_test:',accuracy_test)

accuracy_test: 0.774


In [108]:
y_hattrn = predicttrn(list_matrix0,list_matrix1,count_class)
y_hattrn=np.array(y_hattrn)
y_trn=np.array(y_trn)
accuracy_trn = np.sum(y_trn == y_hattrn)/(5000)
print('accuracy_trn:',accuracy_trn)

accuracy_trn: 0.8442


In [109]:
def confusion_matrix(y,y_hat):
    label= [0 ,1, 2 ,3 ,4 ,5 ,6, 7 ,8 ,9]
    confusionmatrix= np.zeros((10,10))
    for i in range(10):
        for j in range(10):
            confusionmatrix[i][j] = np.sum((y== label[i]) & (y_hat == label[j]))
    return confusionmatrix

In [110]:
confusion_matrix=confusion_matrix(y_tst,y_hat)
confusion_matrix

array([[ 76.,   0.,   1.,   0.,   1.,   5.,   3.,   0.,   4.,   0.],
       [  0., 104.,   1.,   0.,   0.,   2.,   1.,   0.,   0.,   0.],
       [  1.,   3.,  81.,   4.,   2.,   0.,   6.,   1.,   5.,   0.],
       [  0.,   1.,   0.,  80.,   0.,   3.,   2.,   7.,   1.,   6.],
       [  0.,   0.,   1.,   0.,  80.,   1.,   4.,   1.,   2.,  18.],
       [  2.,   1.,   1.,  12.,   3.,  63.,   1.,   1.,   2.,   6.],
       [  1.,   4.,   5.,   0.,   3.,   6.,  70.,   0.,   2.,   0.],
       [  0.,   5.,   4.,   0.,   3.,   0.,   0.,  77.,   3.,  14.],
       [  1.,   1.,   3.,  14.,   3.,   8.,   0.,   1.,  62.,  10.],
       [  1.,   1.,   0.,   3.,   9.,   2.,   0.,   2.,   1.,  81.]])

In [120]:
def precision(label, confusion_matrix):
    c = confusion_matrix[:,label]
    return confusion_matrix[label, label] / c.sum()
    
def recall(label, confusion_matrix):
    row = confusion_matrix[label, :]
    return confusion_matrix[label, label] / row.sum()


In [130]:
print("label precision recall    F1")
for label in range(10):
        p=precision(label,confusion_matrix)
        r=recall(label,confusion_matrix)
        f1 = 2 * (p * r) / (p + r)
        print(f"{label:5d} {p:9.3f} {r:6.3f} {f1:9.3f}")



label precision recall    F1
    0     0.927  0.844     0.884
    1     0.867  0.963     0.912
    2     0.835  0.786     0.810
    3     0.708  0.800     0.751
    4     0.769  0.748     0.758
    5     0.700  0.685     0.692
    6     0.805  0.769     0.787
    7     0.856  0.726     0.786
    8     0.756  0.602     0.670
    9     0.600  0.810     0.689


In [62]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

print(accuracy_score(y_tst,y_hat))
print(recall_score(y_tst,y_hat, average='macro'))
print(precision_score(y_tst,y_hat, average='macro'))
print(f1_score(y_tst,y_hat, average='macro'))

0.774
0.7733848945638426
0.7821993670032373
0.7739639984122183
